In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import gensim
from gensim import corpora, models, similarities
import os
import utility_synopsis
import math
import numpy as np
from scipy.spatial.distance import cosine
import pandas as pd
import matplotlib.pyplot as plt

2017-02-23 14:15:27,994 : INFO : 'pattern' package not found; tag filters are not available for English


In [3]:
def extract_item_info(filename, delimiter, genre_delimiter):
    data = {}

    with open(filename , 'r') as f:
        for i, line in enumerate(f):
            itemId, title, genre_list = map(lambda x: x.strip(), line.split(delimiter))
            
            data[itemId] = (title, genre_list.split(genre_delimiter))
    return data

In [4]:
item_file_name, item_file_delimiter, genre_delimiter = os.path.sep.join(['ml-1m', 'movies.dat']), '::', '|'
item_info = extract_item_info(item_file_name, item_file_delimiter, genre_delimiter)

In [5]:
model_path = '/home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=100_min_count=1_window=1_iter=30.model'

#model = gensim.models.Word2Vec.load('/home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=200_min_count=5_window=2.model' )
model = gensim.models.Word2Vec.load(model_path)

2017-02-23 14:15:32,528 : INFO : loading Word2Vec object from /home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=100_min_count=1_window=1_iter=30.model
2017-02-23 14:15:32,537 : INFO : loading wv recursively from /home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=100_min_count=1_window=1_iter=30.model.wv.* with mmap=None
2017-02-23 14:15:32,538 : INFO : setting ignored attribute syn0norm to None
2017-02-23 14:15:32,538 : INFO : setting ignored attribute cum_table to None
2017-02-23 14:15:32,538 : INFO : loaded /home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=100_min_count=1_window=1_iter=30.model


In [8]:
model.vocab.keys()[:10]

2017-02-23 14:15:46,135 : WARNING : direct access to vocab will not be supported in future gensim releases, please use model.wv.vocab


['2031', '1869', '1868', '643', '344', '345', '346', '347', '340', '341']

In [18]:
print 'most similar:'
for target in model.vocab.keys()[:20]:
    print '*' * 20
    print
    
    neighbors = model.most_similar(target, topn=10)
    
    title, genres = item_info[target]
    print 'target info:'
    print '%50s\t%s' % (title, ', '.join(genres))
    print 'neighbors:'
    for neighbor, simi in neighbors:
        title, genres = item_info[neighbor]
        print '%50s\t%s' % (title, ', '.join(genres))

2017-02-23 14:24:39,142 : WARNING : direct access to vocab will not be supported in future gensim releases, please use model.wv.vocab


********************

target info:
                            $1,000,000 Duck (1971)	Children's, Comedy
neighbors:
                    Barefoot Executive, The (1971)	Children's, Comedy
                       Barenaked in America (1999)	Documentary
                     In the Bleak Midwinter (1995)	Comedy
                                  Beautiful (2000)	Comedy, Drama
                                  Girlfight (2000)	Drama
               Golden Voyage of Sinbad, The (1974)	Action, Adventure
                           Hate (Haine, La) (1995)	Drama
                        Black Cauldron, The (1985)	Animation, Children's
                    Broken Hearts Club, The (2000)	Drama
                                Blank Check (1994)	Children's, Comedy
********************

target info:
                                  Black Dog (1998)	Action, Thriller
neighbors:
                                In Too Deep (1999)	Action, Thriller
                               Big Hit, The (1998)	Action, Come

In [ ]:
'''
********************

target info:
                            $1,000,000 Duck (1971)	Children's, Comedy
neighbors:
                    Barefoot Executive, The (1971)	Children's, Comedy
                       Barenaked in America (1999)	Documentary
                     In the Bleak Midwinter (1995)	Comedy
                                  Beautiful (2000)	Comedy, Drama
                                  Girlfight (2000)	Drama
               Golden Voyage of Sinbad, The (1974)	Action, Adventure
                           Hate (Haine, La) (1995)	Drama
                        Black Cauldron, The (1985)	Animation, Children's
                    Broken Hearts Club, The (2000)	Drama
                                Blank Check (1994)	Children's, Comedy
                                
********************

target info:
                   Clear and Present Danger (1994)	Action, Adventure, Thriller
neighbors:
                                Bitter Moon (1992)	Drama
                                  Committed (2000)	Comedy, Drama
                 City of Lost Children, The (1995)	Adventure, Sci-Fi
                                    Othello (1995)	Drama
                                  Labyrinth (1986)	Adventure, Children's, Fantasy
                              Random Hearts (1999)	Drama, Romance
                                    Copycat (1995)	Crime, Drama, Thriller
                                    Sanjuro (1962)	Action, Adventure
                                     Powder (1995)	Drama, Sci-Fi
                                  Psycho II (1983)	Horror, Thriller
                           
********************

target info:
                                    Beloved (1998)	Drama
neighbors:
                          Elephant Man, The (1980)	Drama
                            Practical Magic (1998)	Drama, Romance
                                 Miss Julie (1999)	Drama
                          Brother's Kiss, A (1997)	Drama
                   Children of a Lesser God (1986)	Drama
                 Children of the Revolution (1996)	Comedy
                                Mighty, The (1998)	Drama
                  Temptress Moon (Feng Yue) (1996)	Romance
                        Ride with the Devil (1999)	Drama, Romance, War
                                   Flawless (1999)	Drama
'''

In [22]:
print 'most dissimilar:'
num_of_most_dissimilar_ones = 10
for target in model.vocab.keys()[:20]:
    print '*' * 20
    print
    
    neighbors = model.most_similar(target, topn=1000000)
    print 'len(neighbors):', len(neighbors)
    neighbors.reverse()
    neighbors = neighbors[:num_of_most_dissimilar_ones]
    #print neighbors
    
    title, genres = item_info[target]
    print 'target info:'
    print '%50s\t%s' % (title, ', '.join(genres))
    print 'neighbors:'
    for neighbor, simi in neighbors:
        title, genres = item_info[neighbor]
        print '%50s\t%s' % (title, ', '.join(genres))

2017-02-23 14:30:05,845 : WARNING : direct access to vocab will not be supported in future gensim releases, please use model.wv.vocab


********************

len(neighbors): 3689
target info:
                            $1,000,000 Duck (1971)	Children's, Comedy
neighbors:
                             Sunchaser, The (1996)	Drama
                               Running Free (2000)	Drama
                With Byrd at the South Pole (1930)	Documentary
                              Outside Ozona (1998)	Drama, Thriller
                                       Anna (1996)	Drama
                        Synthetic Pleasures (1995)	Documentary
                                 Truce, The (1996)	Drama, War
                                      Dingo (1992)	Drama
                                Small Faces (1995)	Drama
City of the Living Dead (Paura nella citt� dei morti viventi) (1980)	Horror
********************

len(neighbors): 3689
target info:
                                  Black Dog (1998)	Action, Thriller
neighbors:
                              Outside Ozona (1998)	Drama, Thriller
                            For Ever Mozart (

In [ ]:
'''
********************

len(neighbors): 3689
target info:
                            $1,000,000 Duck (1971)	Children's, Comedy
neighbors:
                             Sunchaser, The (1996)	Drama
                               Running Free (2000)	Drama
                With Byrd at the South Pole (1930)	Documentary
                              Outside Ozona (1998)	Drama, Thriller
                                       Anna (1996)	Drama
                        Synthetic Pleasures (1995)	Documentary
                                 Truce, The (1996)	Drama, War
                                      Dingo (1992)	Drama
                                Small Faces (1995)	Drama
City of the Living Dead (Paura nella citt� dei morti viventi) (1980)	Horror

********************

len(neighbors): 3689
target info:
                   Clear and Present Danger (1994)	Action, Adventure, Thriller
neighbors:
                                 Truce, The (1996)	Drama, War
                          Project Moon Base (1953)	Sci-Fi
                               Detroit 9000 (1973)	Action, Crime
                        Uninvited Guest, An (2000)	Drama
                                       Anna (1996)	Drama
Blood Spattered Bride, The (La Novia Ensangrentada) (1972)	Horror
                               Running Free (2000)	Drama
                             Sunchaser, The (1996)	Drama
             Dry Cleaning (Nettoyage � sec) (1997)	Drama
                With Byrd at the South Pole (1930)	Documentary
             
********************

len(neighbors): 3689
target info:
                                    Beloved (1998)	Drama
neighbors:
             Dry Cleaning (Nettoyage � sec) (1997)	Drama
                        Uninvited Guest, An (2000)	Drama
                          Project Moon Base (1953)	Sci-Fi
                              Outside Ozona (1998)	Drama, Thriller
                With Byrd at the South Pole (1930)	Documentary
                             Sunchaser, The (1996)	Drama
                                 Truce, The (1996)	Drama, War
Paralyzing Fear: The Story of Polio in America, A (1998)	Documentary
                            For Ever Mozart (1996)	Drama
Jeanne and the Perfect Guy (Jeanne et le gar�on formidable) (1998)	Comedy, Romance
'''